# Make .mat Files

The TensorFlow implementation of FBPConvNet takes .mat files for training and testing. This notebook makes these files from the S70 dataset. There are 50 images in the test set and 644 images in the training set. The reconstructed images in the training set are sparse with 60 views.

### Import Libraries

In [30]:
import os
import numpy as np
import fbpconv
import scipy.io as sio
import matplotlib.pyplot as plt
from scipy.misc import imsave, imread
from skimage.transform import radon, iradon
from random import shuffle

### Make reconstructed views

In [32]:
num_views = 60

cur_dir = os.getcwd()
data_dir = os.path.join(cur_dir, 'data/train/gt')
train_dir = os.path.join(cur_dir, 'data/VHF_dataset/train/')
test_dir = os.path.join(cur_dir, 'data/VHF_dataset/test/')
list_dir = os.listdir(data_dir)
shuffle(list_dir)
i = 0
for file in list_dir:
    # Read in image
    path = os.path.join(data_dir, file)
    ct = np.load(path)
    
    # Have 50 images be in test set. The rest are in train set.
    if i < 50:
        dest_dir = test_dir
    else:
        dest_dir = train_dir
    
    imsave(dest_dir + file[:-6] + 'gt.png', ct)
    
    # Make sparse sinograms
    theta = np.linspace(0, 180, num_views, endpoint = False) 
    sino = radon(ct, theta=theta)

    # Reconstruct image and save
    recon = iradon(sino, theta=theta)
    imsave(dest_dir + file[:-6] + 'fbp.png', recon)    
    
    i += 1
    

c:\users\daniel\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
c:\users\daniel\appdata\local\programs\python\python36\lib\site-packages\skimage\transform\radon_transform.py:58: UserWarning: The default of `circle` in `skimage.transform.radon` will change to `True` in version 0.15.
  warn('The default of `circle` in `skimage.transform.radon` '
c:\users\daniel\appdata\local\programs\python\python36\lib\site-packages\skimage\transform\radon_transform.py:204: UserWarning: The default of `circle` in `skimage.transform.iradon` will change to `True` in version 0.15.
  warn('The default of `circle` in `skimage.transform.iradon` '
c:\users\daniel\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and wi

### Make the .mat files

In [33]:
# Turn GT and recon images into numpy arrays for .mat file
cur_dir = os.getcwd()
sets = ['train', 'test']

for d_set in sets:
    data_dir = os.path.join(cur_dir, 'data/VHF_dataset/' + d_set)
    recon_imgs = []
    gt_imgs = []
    for file in os.listdir(data_dir):
        path = os.path.join(data_dir, file)
        img = imread(path)

        if file.endswith('gt.png'):
            gt_imgs.append(img)
        elif file.endswith('fbp.png'):
            recon_imgs.append(img)

    recon_imgs = np.asarray(recon_imgs)
    gt_imgs = np.asarray(gt_imgs)

    # Move axes around to get into correct format for FBPConvNet
    recon_imgs = np.moveaxis(recon_imgs[:,:,:, np.newaxis], 0, -1)
    gt_imgs = np.moveaxis(gt_imgs[:,:,:, np.newaxis], 0, -1)

    print(recon_imgs.shape)
    data_dict = {}
    data_dict['sparse'] = recon_imgs
    data_dict['label'] = gt_imgs
    sio.savemat(d_set + '_vhf.mat', data_dict)



c:\users\daniel\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  # This is added back by InteractiveShellApp.init_path()


(512, 512, 1, 733)
(512, 512, 1, 50)


In [ ]:
mat_contents= sio.loadmat('train_elips.mat')
print(mat_contents.keys())
print(mat_contents['sparse'].shape)


In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(121)
plt.imshow(mat_contents['sparse'][:,:,0,0], cmap = plt.cm.bone)
plt.subplot(122)
plt.imshow(mat_contents['label'][:,:,0,0], cmap = plt.cm.bone)

In [ ]:
train_dir = os.path.join(dest_dir, 'train')
dtrain_dir = os.path.join(dest_dir, 'dtrain')
test_dir = os.path.join(dest_dir, 'test')
dtest_dir = os.path.join(dest_dir, 'dtest')
_load_whole_data('fbpconv', train_dir, dtrain_dir)